# Story Generation
We remember things better as stories. The plan here is to pick phrases from a pack we have learnt, and incoprate them into a story.

In [1]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.llm_tools.story_generation import generate_story
from src.phrases.search import get_phrases_by_collection
from src.story import Story
from src.utils import save_json, load_json

In [2]:
COLLECTION = "LM2000"
DECK = "Pack02"

In [3]:
phrase_list = get_phrases_by_collection(COLLECTION, DECK)

(y) Authenticated with Google Cloud project: swedish-course


In [4]:
story_name, summary, story_dialogue = generate_story(phrase_list=phrase_list)

In [5]:
print(story_name)  #
print(summary)

The Garden Interview
Alex helps Sam prepare for an important job interview while they work in Sam's garden, offering advice and support through a mishap and celebration.


In [6]:
story_name = "The Garden Interview"
summary = "Alex helps Sam prepare for an important job interview while they work in Sam's garden, offering advice and support through a mishap and celebration."

In [6]:
save_json(story_dialogue, f"../data/stories/{COLLECTION}-{DECK}-{story_name}-story.json")

In [5]:
story_dialogue = load_json(f"../data/stories/{COLLECTION}-{DECK}-{story_name}-story.json")

In [7]:
story = Story.create(
    title=story_name,
    summary=summary,
    story_dialogue=story_dialogue,
    collection=COLLECTION,
    deck=DECK,
)

(y) Authenticated with Google Cloud project: swedish-course
(y) Natural Language API client initialized


In [8]:
story.translate("sv-SE", refine=True)
story.upload()
story.publish_story("sv-SE")

(y) Google Translate API client initialized
2026-01-10 23:06:59 - audio-language-trainer - WARNING - nlp.py:42 - analyze_text_syntax: language not supported by Google NLP: sv
2026-01-10 23:06:59 - audio-language-trainer - ERROR - nlp.py:154 - API Tokenization failed for sv
Traceback (most recent call last):
  File "c:\Users\andyl\Python\audio-language-trainer\src\nlp.py", line 32, in analyze_text_syntax
    response = client.analyze_syntax(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andyl\Python\audio-language-trainer\.venv\Lib\site-packages\google\cloud\language_v1\services\language_service\client.py", line 1139, in analyze_syntax
    response = rpc(
               ^^^^
  File "c:\Users\andyl\Python\audio-language-trainer\.venv\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andyl\Python\audio-language-trainer\.venv\Lib\site-packages\google\api_co

'https://storage.googleapis.com/audio-language-trainer-stories/stories/en-GB/sv-SE/the_garden_interview_fb9ad9/index.html'